# 🔧 Fine-Tuning `thenlper/gte-base` with Basic Hyperparameter Tuning
This notebook fine-tunes `gte-base` using tweet ↔ paper pairs and tests different batch size and epoch configurations.

In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import pandas as pd
import pickle

# Load data
train_df = pd.read_csv("subtask4b_query_tweets_train.tsv", sep="\t", names=["post_id", "tweet_text", "cord_uid"])
with open("subtask4b_collection_data.pkl", "rb") as f:
    papers_df = pickle.load(f)
papers_df["text"] = papers_df["title"] + ". " + papers_df["abstract"]
paper_dict = dict(zip(papers_df["cord_uid"], papers_df["text"]))

# Clean text
def clean(text):
    return text.lower().strip()

# Prepare training examples (limit to 500 for speed)
train_samples = []
for _, row in train_df.iterrows():
    if row["cord_uid"] in paper_dict:
        tweet = clean(row["tweet_text"])
        paper = clean(paper_dict[row["cord_uid"]])
        train_samples.append(InputExample(texts=[tweet, paper]))

train_samples = train_samples[:500]  # limit AFTER collecting

print(f"✅ Prepared {len(train_samples)} training pairs")


/Users/gokayyil/checkthatenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/2r/b056w9_x2r7_hv60_h769b180000gn/T/ipykernel_20975/3901362259.py:9: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  papers_df = pickle.load(f)


✅ Prepared 500 training pairs


In [ ]:
# Grid search configs
configs = [
    {"batch_size": 16, "epochs": 3},
    {"batch_size": 16, "epochs": 4},
    {"batch_size": 32, "epochs": 4},
    {"batch_size": 32, "epochs": 5}
]

for config in configs:
    model = SentenceTransformer("thenlper/gte-base")
    dataloader = DataLoader(train_samples, shuffle=True, batch_size=config["batch_size"])
    loss_fn = losses.MultipleNegativesRankingLoss(model)
    print(f"\n🚀 Training b={config['batch_size']} e={config['epochs']}")
    model.fit(
        train_objectives=[(dataloader, loss_fn)],
        epochs=config["epochs"],
        warmup_steps=100,
        show_progress_bar=True
    )
    model.save(f"fine_tuned_gte_b{config['batch_size']}_e{config['epochs']}")
    print("✅ Saved model.")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/gokayyil/checkthatenv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/gokayyil/checkthatenv/lib/python3.10/site-packages/traitlets/config/appl


🚀 Training b=16 e=3


Epoch: 100%|█████████████████████████████████████| 3/3 [49:39<00:00, 993.23s/it]


✅ Saved model.

🚀 Training b=16 e=4


Epoch: 100%|██████████████████████████████████| 4/4 [1:09:34<00:00, 1043.69s/it]


✅ Saved model.

🚀 Training b=32 e=4


Iteration:   0%|                                         | 0/16 [00:00<?, ?it/s]